In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import scipy.io as sio
import numpy as np

In [0]:
scipy_sio = sio.loadmat('/content/drive/My Drive/Colab Notebooks/cars_annos.mat', squeeze_me = True)

In [0]:
scipy_sio

In [0]:
#extract class_names
class_name = scipy_sio['class_names']
class_name[0:10]
# has linkage to annotation['class']

In [0]:
#extract annotations, contains 7 variables
annotation = scipy_sio['annotations']
annotation[0:5]
# 7 values:
# 0: relative_im_path:  image location & name
# 1: bbox_x1:           Min x-value of the bounding box, in pixels
# 2: bbox_y1:           Min y-value of the bounding box, in pixels
# 3: bbox_x2:           Max x-value of the bounding box, in pixels
# 4: bbox_y2:           Max y-value of the bounding box, in pixels
# 5: class:             Target model, ex: img 000001-000003 have class = 1, so it is 'AM General Hummer SUV 2000'
# 6: test:              test = 1, train = 0


In [0]:
#convert annotation [array of tuples] to 1D array
annotation_array = [a for b in annotation for a in b]
annotation_array[0:14]

In [0]:
#convert 1D array into 7d array/matrix
annotation_matrix = np.reshape(annotation_array, (-1,7))
#annotation_matrix = annotation_matrix.tolist()    #remove dtype
annotation_matrix[0:5]

In [0]:
# ex: extract class variable
tgt_class = annotation_matrix[:,5]
tgt_class = tgt_class.tolist()

In [0]:
len(np.unique(tgt_class))
#there are 196 unique classes in annotation matrix

In [0]:
len(np.unique(class_name))
#there are 196 different classes in array class_name